In [61]:
from client_info import user_connection, token, usr_con_stronger
import pandas as pd
import sqlalchemy as sa
import spotipy
import requests
import math
import numpy as np
from snowflake_con_manager import USER,PASSWORD,ACCOUNT,DATABASE,SCHEMA,ROLE,WAREHOUSE
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector

In [49]:
conn = snowflake.connector.connect(
        user=USER
        ,password=PASSWORD
        ,account=ACCOUNT
        ,warehouse=WAREHOUSE
        ,database=DATABASE
        ,schema=SCHEMA
        ,role=ROLE)

In [63]:
user = user_connection()
# playlists = user.current_user_playlists()

In [59]:
test2= user.current_user_recently_played(before=1675742400000)

In [57]:
test2

{'items': [{'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0GM7qgcRCORpGnfcN2tCiB'},
       'href': 'https://api.spotify.com/v1/artists/0GM7qgcRCORpGnfcN2tCiB',
       'id': '0GM7qgcRCORpGnfcN2tCiB',
       'name': 'Tainy',
       'type': 'artist',
       'uri': 'spotify:artist:0GM7qgcRCORpGnfcN2tCiB'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
      'MY',
      'MT',
      'MX',
      'NL',
      'NZ',
      'NI',
      'NO',
      'PA',
      'PY',
      'PE',
      'PH',
      'PL',
      'PT',
      'SG',
      'SK',
      'ES',
      'SE',
      'CH

In [130]:
def get_user_recent_tracks(connection:spotipy.Spotify,search_date:str):
    from datetime import date
    from datetime import datetime
    import time
    param_date = date.fromisoformat(search_date)
    date_in_unix = int(time.mktime(param_date.timetuple()) * 1000)
    recent = connection.current_user_recently_played(limit=50,after=date_in_unix)
    all_fields = []
    for i in range(len(recent)):
        fields = {
            'track_id': recent['items'][i]['track']['id']
            ,'track_name': recent['items'][i]['track']['name']
            ,'artist_name': ' - '.join([recent['items'][i]['track']['artists'][j]['name'] for j in range(len(recent['items'][i]['track']['artists']))])
            ,'from':datetime.utcfromtimestamp(int(recent['cursors']['before'])/1000).strftime('%Y-%m-%d %H:%M:%S')
            ,'to': datetime.utcfromtimestamp(int(recent['cursors']['after'])/1000).strftime('%Y-%m-%d %H:%M:%S')

        }
        all_fields.append(fields)
    return all_fields


5


: 

In [22]:
print(playlists.keys())
tracks = dict()
for k in playlists['items']:
    tracks.update({k['name']:k['id']})
    

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])


In [23]:
# print(tracks)
print(tracks['Jazz'])

1JvsSfXPZh4NY5phCjd3G1


In [16]:
playlist_tracks = user.playlist_tracks("1JvsSfXPZh4NY5phCjd3G1",limit=100,offset=200)


In [24]:
# playlist_tracks['items'][27]['track']['name']

def get_amount_tracks(total:int):
    if total <= 0:
        return []
    else:
        result = []
        whole = math.ceil((int(total))/100)
        decimal = (total/100) - (int(total))//100
        for i in range(whole):
            result.append((i*100))
        if decimal:
            result.append(math.ceil(decimal*100))
        result[-1] = result[-2]+result[-1]
        return result
    
test = get_amount_tracks(228)
print(test)
# print(test1)

[0, 100, 200, 228]


In [25]:
def get_tracks(playlist_id:str):
    table = list()
    counter = 0
    try:
        playlist_tracks = user.playlist_tracks(playlist_id,limit=100)
        rounds = get_amount_tracks(int(playlist_tracks['total']))
        for i in rounds[:-1]:
            playlist_tracks = user.playlist_tracks(playlist_id,offset=i)
            for k in range(0,len(range(i,rounds[counter+1]))-1):
                track = playlist_tracks['items'][k]['track'].items()
                track_dict = {item[0]: item[1] for item in track}
                table.append(track_dict)
                table[k]['playlist_id'] = playlist_id
                table[k]['track_link'] = playlist_tracks['items'][k]['track']['external_urls']['spotify']
                table[k]['added_at'] = playlist_tracks['items'][k]['added_at']
                table[k]['added_by'] = playlist_tracks['items'][k]['added_by']['id']
            counter += 1
    except Exception as e:
        pass
    return table


In [26]:
table = []
for k,v in tracks.items():
    table += get_tracks(v)

In [30]:
df = pd.DataFrame(table)

df = df[['playlist_id','id','name','popularity','duration_ms','track_number','added_at','added_by','explicit','available_markets','disc_number','track_link']]
df['track_id'] = df['id']
df['track_name'] = df['name']
df = df[['playlist_id','track_id','track_name','track_number','added_at','added_by','track_link']]
df.head(1)

,playlist_id,track_id,track_name,track_number,added_at,added_by,track_link
0,48OozGbCyax8Y5BkWlwLpx,6n3HGiq4v35D6eFOSwqYuo,Teenage Fever,14,2023-04-27T01:14:20Z,vanessarosso26,https://open.spotify.com/track/6n3HGiq4v35D6eF...


In [128]:
write_pandas(conn,df,table_name='RAW_PLAYLISTS',quote_identifiers=False)

(True,
 1,
 3131,
 [('jyaxuggrtt/file0.txt',
   'LOADED',
   3131,
   3131,
   1,
   0,
   None,
   None,
   None,
   None)])

In [4]:
tracks = pd.read_sql_query(con=conn,sql='SELECT distinct track_id FROM raw_playlists')



C:\Users\pavel\AppData\Local\Temp\ipykernel_29852\749052625.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tracks = pd.read_sql_query(con=conn,sql='SELECT distinct track_id FROM raw_playlists')


In [ ]:
tracks['TRACK_ID'][5]


'152lZdxL1OR0ZMW6KquMif'

In [28]:
# track_list = list(tracks['TRACK_ID'])
# len(test)

# tracklist_len = get_amount_tracks(len(test))
# list_append = []
# counter = 0

def get_track_info(track_id):

    user_track = user.track(track_id)
    get_genre = user.artist(user_track['artists'][0]['id'])
    fields = {
         'track_id'     :        user_track['id']
        ,'track_name'   :     user_track['name']
        ,'track_genre'  :    get_genre['genres']
        ,'duration'     :        user_track['duration_ms']
        ,'popularity'   :      user_track['popularity']
        ,'track_number' :    user_track['track_number']
        ,'artist_id'    :       user_track['artists'][0]['id']
        ,'artist_name'  :     user_track['artists'][0]['name']
    }
    return fields



In [7]:
list_append = []
list_append += [get_track_info(i) for i in tracks['TRACK_ID'][:100]]


In [15]:
dftest = pd.DataFrame(list_append)


In [16]:
dftest

,track_id,track_name,track_genre,duration,popularity,track_number,artist_id,artist_name
0,6n3HGiq4v35D6eFOSwqYuo,Teenage Fever,"[canadian hip hop, canadian pop, hip hop, pop ...",219692,84,14,3TVXtAsR1Inumwj472S9r4,Drake
1,7JXZq0JgG2zTrSOAgY8VMC,Jungle,"[canadian hip hop, canadian pop, hip hop, pop ...",320400,78,16,3TVXtAsR1Inumwj472S9r4,Drake
2,7zFXmv6vqI4qOt4yGf3jYZ,Get You (feat. Kali Uchis),[canadian contemporary r&b],278179,85,1,20wkVLutqVOYrc0kxFs7rA,Daniel Caesar
3,1boXOL0ua7N2iCOUVI1p9F,Japanese Denim,[canadian contemporary r&b],270846,82,2,20wkVLutqVOYrc0kxFs7rA,Daniel Caesar
4,6PGoSes0D9eUDeeAafB2As,LOVE. FEAT. ZACARI.,"[conscious hip hop, hip hop, rap, west coast rap]",213400,86,10,2YZyLoL8N0Wb9xBt1NhZWg,Kendrick Lamar
...,...,...,...,...,...,...,...,...
95,58VNpcJpI5dEKxYByCkbXc,Miss Summer,[canadian contemporary r&b],162376,69,1,5aMIbwZQvP2MHPMVC5zCGj,ODIE
96,3XlzpaHje8MlpaGe8twgr7,A Cappella,[canadian contemporary r&b],88916,59,4,20wkVLutqVOYrc0kxFs7rA,Daniel Caesar
97,7FAFkQQZFeNwOFzTrSDFIh,The Spins,"[hip hop, pittsburgh rap, pop rap, rap]",195873,84,6,4LLpKhyESsyAXpc4laK94U,Mac Miller
98,410ZZP746AQeiywhKvXWCo,SORRY NOT SORRY,"[hip hop, rap]",206049,74,24,4V8LLVI7PbaPR0K2TGSxFF,"Tyler, The Creator"


In [ ]:

import json
for _,row in dftest.iterrows():
    row['track_genre'] = json.dumps(row['track_genre'])
    print(tuple(row[col] for col in dftest.columns))

In [70]:
for i in dftest.columns:
    if dftest[i].apply(lambda x: isinstance(x, list)).any():
        dftest[i] = dftest[i].apply(lambda x: '-'.join(x))

print(dftest['track_genre'])

0     canadian hip hop-canadian pop-hip hop-pop rap-rap
1     canadian hip hop-canadian pop-hip hop-pop rap-rap
2                             canadian contemporary r&b
3                             canadian contemporary r&b
4          conscious hip hop-hip hop-rap-west coast rap
                            ...                        
95                            canadian contemporary r&b
96                            canadian contemporary r&b
97                   hip hop-pittsburgh rap-pop rap-rap
98                                          hip hop-rap
99    alternative rock-art rock-melancholia-oxford i...
Name: track_genre, Length: 100, dtype: object


In [ ]:
for _,row in dftest.iterrows():
    print(tuple(row[col] for col in dftest.columns))

In [60]:
for _, row in dftest.iterrows():
    print((', '.join([f"row[{i}]" for i in dftest.columns])))

row[track_id], row[track_name], row[track_genre], row[duration], row[popularity], row[track_number], row[artist_id], row[artist_name]


In [32]:
def write_to_db(conn:snowflake.connector,table:str,df:pd.DataFrame,if_exists='replace'):
    new_df = list(zip(df.columns,df.dtypes))
    new_df = [(i[0],'string') if i[1] =='object' else (i[0],i[1]) for i in new_df]
    new_df = [(i[0],'int') if 'int' in str(i[1]) else (i[0],i[1]) for i in new_df]
    new_df = [(i[0],'float') if 'float' in str(i[1]) else (i[0],i[1]) for i in new_df]
    columns = [str(i[0])+' '+str(i[1])+'\n' for i in new_df]
    script = f"""
    CREATE TABLE {table}
    (
    {','.join(columns)}
    );
    """
    run = False
    cursor = conn.cursor()
    if if_exists == 'replace':
        cursor.execute(f"DROP TABLE {table}")
        cursor.execute(script)
    else:
        for i in df.columns:
            if df[i].apply(lambda x: isinstance(x, list)).any():
                df[i] = df[i].apply(lambda x: '-'.join(x))
                
        for _, row in df.iterrows():
            query = f"INSERT INTO {table} ({', '.join(df.columns)}) VALUES ({', '.join(['%s' for _ in df.columns])})"
            values = tuple(row[col] for col in df.columns)
            cursor.execute(query,values)
            run = True
    try:
        if run:
            pass
        else:
            write_pandas(conn,df,table_name=table,quote_identifiers=False)
            run = True
    except Exception as e:
            if run:
                pass
            else:
                write_pandas(conn,df,table_name=table,quote_identifiers=True)
                run = True 
    finally:
        if run:
            print("Write Completed")
        else:

            print(str(e)+"Check inputs and code")

# write_to_db(conn,'raw_tracks',dftest,'append')

In [33]:
write_to_db(conn=conn,table='RAW_PLAYLISTS',df=df,if_exists='replace')

Write Completed


In [246]:
write_pandas(conn,dftest,table_name='raw_tracks',quote_identifiers=False)

(True,
 1,
 100,
 [('sxmoigfnlo/file0.txt', 'LOADED', 100, 100, 1, 0, None, None, None, None)])